In [23]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint


In [25]:
# Parameters
IMAGE_SIZE = (224, 224)
BATCH_SIZE = 32
EPOCHS = 20
DATASET_PATH = r"D:\Eye Scan\DATASET_101"  # Update this path


In [27]:
# Data Generator
datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

train_data = datagen.flow_from_directory(
    DATASET_PATH,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training',
    shuffle=True
)

val_data = datagen.flow_from_directory(
    DATASET_PATH,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation'
)



Found 7861 images belonging to 2 classes.
Found 1964 images belonging to 2 classes.


In [29]:
# Build Model
base_model = ResNet50(include_top=False, weights='imagenet', input_shape=(224, 224, 3))
base_model.trainable = False

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(train_data.num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

model.compile(optimizer=Adam(0.0001), loss='categorical_crossentropy', metrics=['accuracy'])


In [31]:
# Callbacks
callbacks = [
    EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=1e-6),
    ModelCheckpoint("final_fundus_model.h5", save_best_only=True, monitor='val_accuracy')
]


In [33]:
# Train
model.fit(train_data, validation_data=val_data, epochs=EPOCHS, callbacks=callbacks)
model.save("final_fundus_model.h5")
print("✅ Model training complete and saved.")


Epoch 1/20
246/246 [==============================] - ETA: 0s - loss: 0.4576 - accuracy: 0.8480

C:\Users\ajain\anaconda3\envs\testenv\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


246/246 [==============================] - 653s 3s/step - loss: 0.4576 - accuracy: 0.8480 - val_loss: 0.4091 - val_accuracy: 0.8579 - lr: 1.0000e-04
Epoch 2/20
246/246 [==============================] - 655s 3s/step - loss: 0.4272 - accuracy: 0.8579 - val_loss: 0.4103 - val_accuracy: 0.8579 - lr: 1.0000e-04
Epoch 3/20
246/246 [==============================] - 651s 3s/step - loss: 0.4180 - accuracy: 0.8579 - val_loss: 0.4096 - val_accuracy: 0.8579 - lr: 1.0000e-04
Epoch 4/20
246/246 [==============================] - 753s 3s/step - loss: 0.4152 - accuracy: 0.8579 - val_loss: 0.4077 - val_accuracy: 0.8579 - lr: 1.0000e-04
Epoch 5/20
246/246 [==============================] - 1002s 4s/step - loss: 0.4130 - accuracy: 0.8579 - val_loss: 0.4084 - val_accuracy: 0.8579 - lr: 1.0000e-04
Epoch 6/20
246/246 [==============================] - 997s 4s/step - loss: 0.4115 - accuracy: 0.8579 - val_loss: 0.4080 - val_accuracy: 0.8579 - lr: 1.0000e-04
Epoch 7/20
246/246 [==============================

In [43]:
# Unfreeze last few layers of ResNet50 for fine-tuning
for layer in model.layers[-30:]:
    if not isinstance(layer, tf.keras.layers.BatchNormalization):
        layer.trainable = True

# Re-compile with a lower learning rate
model.compile(optimizer=Adam(1e-5), loss='categorical_crossentropy', metrics=['accuracy'])

# Fine-tune with fewer epochs
fine_tune_epochs = 5
model.fit(train_data, validation_data=val_data, epochs=fine_tune_epochs, callbacks=callbacks)

# Save fine-tuned model
model.save("final_fundus_model_finetuned.keras")


Epoch 1/5
246/246 [==============================] - 825s 3s/step - loss: 0.4128 - accuracy: 0.8579 - val_loss: 0.4107 - val_accuracy: 0.8579 - lr: 1.0000e-05
Epoch 2/5
246/246 [==============================] - 868s 4s/step - loss: 0.4132 - accuracy: 0.8579 - val_loss: 0.4132 - val_accuracy: 0.8579 - lr: 1.0000e-05
Epoch 3/5
246/246 [==============================] - 843s 3s/step - loss: 0.4132 - accuracy: 0.8579 - val_loss: 0.4080 - val_accuracy: 0.8579 - lr: 1.0000e-05
Epoch 4/5
246/246 [==============================] - 1258s 5s/step - loss: 0.4138 - accuracy: 0.8579 - val_loss: 0.4080 - val_accuracy: 0.8579 - lr: 1.0000e-05
Epoch 5/5
246/246 [==============================] - 1277s 5s/step - loss: 0.4118 - accuracy: 0.8579 - val_loss: 0.4089 - val_accuracy: 0.8579 - lr: 1.0000e-05


In [51]:
import tensorflow as tf

# Load the model
model = tf.keras.models.load_model("final_fundus_model_finetuned.keras")

# Display model summary
model.summary()

# Check output layer info
output_layer = model.layers[-1]
print("Output layer activation:", output_layer.activation.__name__)
print("Number of output units (classes):", output_layer.output_shape[-1])


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 conv1_pad (ZeroPadding2D)   (None, 230, 230, 3)          0         ['input_1[0][0]']             
                                                                                                  
 conv1_conv (Conv2D)         (None, 112, 112, 64)         9472      ['conv1_pad[0][0]']           
                                                                                                  
 conv1_bn (BatchNormalizati  (None, 112, 112, 64)         256       ['conv1_conv[0][0]']          
 on)                                                                                          

In [1]:
import tkinter as tk
from tkinter import filedialog, messagebox
from PIL import Image, ImageTk
import numpy as np
import tensorflow as tf
from fpdf import FPDF
import os

# Load model
model = tf.keras.models.load_model("final_fundus_model_finetuned.keras")

# Label map
label_mapping = {
    0: ("Cataract", "Clouding of the lens causing decreased vision."),
    1: ("Glaucoma", "Damage to the optic nerve usually due to high pressure."),
    2: ("Hypertensive Retinopathy", "Retinal damage from high blood pressure."),
    3: ("Macular Degeneration", "Degeneration of the central retina (macula)."),
    4: ("Normal", "The eye appears to be healthy."),
    5: ("Retinopathy", "Damage to the retina often associated with diabetes.")
}

def preprocess_image(path):
    img = tf.keras.preprocessing.image.load_img(path, target_size=(224, 224))
    img_array = tf.keras.preprocessing.image.img_to_array(img)
    return np.expand_dims(img_array, axis=0) / 255.0

def predict_image(path):
    img = preprocess_image(path)
    preds = model.predict(img)
    predicted_class = np.argmax(preds)
    return label_mapping.get(predicted_class, ("Unknown", "No description available."))

def generate_pdf(name, sex, phone, result, description):
    pdf = FPDF()
    pdf.add_page()
    pdf.set_font("Arial", size=14)
    pdf.cell(200, 10, txt="Eye Disease Prediction Report", ln=True, align="C")
    pdf.ln(10)
    pdf.cell(200, 10, txt=f"Name: {name}", ln=True)
    pdf.cell(200, 10, txt=f"Sex: {sex}", ln=True)
    pdf.cell(200, 10, txt=f"Phone: {phone}", ln=True)
    pdf.cell(200, 10, txt=f"Result: {result}", ln=True)
    pdf.multi_cell(0, 10, txt=f"Description: {description}")
    pdf.output("eye_disease_report.pdf")
    messagebox.showinfo("PDF Report", "Report saved as 'eye_disease_report.pdf'.")

# GUI Setup
root = tk.Tk()
root.title("Eye Disease Classifier")

name_var = tk.StringVar()
sex_var = tk.StringVar()
phone_var = tk.StringVar()
result_var = tk.StringVar()
desc_var = tk.StringVar()
img_path = None

def upload_image():
    global img_path
    img_path = filedialog.askopenfilename(filetypes=[("Image Files", "*.png *.jpg *.jpeg")])
    if img_path:
        img = Image.open(img_path)
        img = img.resize((200, 200))
        img_tk = ImageTk.PhotoImage(img)
        image_label.configure(image=img_tk)
        image_label.image = img_tk

def run_prediction():
    if not img_path:
        messagebox.showerror("Error", "Please upload an image.")
        return
    result, description = predict_image(img_path)
    result_var.set(result)
    desc_var.set(description)

def save_report():
    name = name_var.get()
    sex = sex_var.get()
    phone = phone_var.get()
    if not all([name, sex, phone, result_var.get()]):
        messagebox.showerror("Missing Info", "Fill all fields and run prediction first.")
        return
    generate_pdf(name, sex, phone, result_var.get(), desc_var.get())

# Layout
tk.Label(root, text="Name:").grid(row=0, column=0, sticky='e')
tk.Entry(root, textvariable=name_var).grid(row=0, column=1)

tk.Label(root, text="Sex:").grid(row=1, column=0, sticky='e')
tk.Entry(root, textvariable=sex_var).grid(row=1, column=1)

tk.Label(root, text="Phone:").grid(row=2, column=0, sticky='e')
tk.Entry(root, textvariable=phone_var).grid(row=2, column=1)

tk.Button(root, text="Upload Image", command=upload_image).grid(row=3, column=0, columnspan=2, pady=5)

image_label = tk.Label(root)
image_label.grid(row=4, column=0, columnspan=2)

tk.Button(root, text="Predict", command=run_prediction).grid(row=5, column=0, columnspan=2, pady=5)

tk.Label(root, text="Result:").grid(row=6, column=0, sticky='e')
tk.Label(root, textvariable=result_var).grid(row=6, column=1, sticky='w')

tk.Label(root, text="Description:").grid(row=7, column=0, sticky='ne')
tk.Label(root, textvariable=desc_var, wraplength=300, justify="left").grid(row=7, column=1, sticky='w')

tk.Button(root, text="Generate PDF Report", command=save_report).grid(row=8, column=0, columnspan=2, pady=10)

root.mainloop()


1/1 [==============================] - 0s 188ms/step
